# Garnet fractionation

Basically the same set-up as the previous garnet fractionation example, but here used to show:

1. New scripting in tc350beta3

2. New composition-tracking

In [1]:
import tawnycalc as tc
import os

### Keeping track of mass and energy as the system gains or loses phases

In a THERMOCALC calculation (a call to context.execute()), THERMOCALC works with a chemical system defined as a vector of the number of oxide units present, normalised to 100. If the user chooses to specify this chemical system (the "bulk composition") via a block of rbi scripts, THERMOCALC constructs the vector of oxide units from the rbi scripts, but makes no other use of the rbi information. For defining the equilibrium system, THERMOCALC is only interested in the relative proportions of oxide units.

TawnyCALC, however, handles systems in which chemical elements may enter or leave the system, and be fractionated from each other. We therefore have to keep track of the number of oxide units entering and leaving the system.

A vector of oxide units can be converted into a scalar, the number of atoms, by taking its dot product with a vector of the number of atoms in each oxide unit (there are 3 atoms in SiO2, 5 atoms in Al2O3, ....). This is useful in particular because THERMOCALC outputs the extensive properties of individual phases (modes, energies, etc) on a per-atom basis.

In [2]:
# First we must construct a dictionary, oxinfo, of atoms per oxide, 
# from the 'datasets' file 'oxide_data.txt'.

oxidedata=os.path.join(os.path.dirname(tc.__file__),"datasets/oxide_data.txt")
with open(oxidedata) as lst:
    oxdat=lst.readlines()

oxdat = [ l.split() for l in oxdat ]

ox   = [ l for l in oxdat if (len(l)>0 and l[0]=='oxides'  ) ][0][1:]
lAox = [ l for l in oxdat if (len(l)>0 and l[0]=='no_atoms') ][0][1:]
nAox = [ int(a) for a in lAox]

oxinfo = { ox[i] : nAox[i] for i in range(len(ox)) }

In [3]:
def atoms_from_ox(oxlist):
    """
    Calculate the total number of atoms implied by a list of 
    numbers of oxide units
    """
    return sum([ x * y for x, y in zip(oxlist,nAox) ])
    
    
    
def system_components(oxides,rbi,phases):    
    """
    Calculate the number of oxide units and total number of 
    atoms in a specified subset of the phases in an rbi construct.

    Params
    ------
        oxides: list
            list of oxide names as strings.
        rbi: tawnycalc.rbi
            rbi construct.>
        phases: list
            list of names of phases from the rbi construct 
            as strings; OR, "all" for whole rbi block
    Returns
    -------
        [total no. atoms, [ox1, ox2, ...]]
    """
    
    # ensure modes in entire rbi block are normalised
    # (true for THERMOCALC output but not necessarily for user input)
    phases0 = list(rbi.keys())  # all phases in rbi object
    modes0 = [ rbi[phase]["mode"] for phase in phases0 ]
    modes0N = [ m/sum(modes0) for m in modes0 ]
        
    # for each phase in the specified subset, find the numbers of
    # oxide units contributed to the system
    oxlists = []   
    modes = []
    if phases == 'all':
        phase_subset = phases0
    else:
        phase_subset = phases
        
    for phase in phase_subset:
        pos = phases0.index(phase)
        mode = modes0N[pos]   # no. atoms of phase = mode
        modes.append(mode)
        oxx = [ rbi[phase][x] for x in oxides ]
        apfu = atoms_from_ox(oxx)      # atoms per formula unit
        oxf = [ mode * x / apfu for x in oxx ] 
        oxlists.append(oxf)
    
    # for each oxide, sum the contributions from each of the phases
    # in the specified subset, and calculate the total number of atoms
    oxlistsT = [ [oxlists[j][i] for j in range(len(oxlists))] for i in range(len(oxlists[0])) ]
    nOxsubset = [ sum(l) for l in oxlistsT ]
    sum_of_modes = sum(modes)

    return [sum_of_modes, nOxsubset]


#system_components(oxlist,results.rbi,list(removal_fraction.keys()))
# results.rbi

### Setting up and scripting problem

In [4]:
# set up lists of possible phases
phases_all = "chl bi pa ru g ilmm sph"

# set up path taken and phases added/removed
removal_fraction = {}
removal_fraction["g"] = 1

PT_steps = []
PT_steps.append( (11.0, 600 ) )
PT_steps.append( (12.0, 630 ) )
PT_steps.append( (12.2, 650 ) )
PT_steps.append( (11.5, 670 ) )
PT_steps.append( (10.0, 660 ) )

In [5]:
# To start with a clean context, set `scripts_dir=None`
context = tc.Context(scripts_dir=None)
context.prefs

calcmode   : 1
scriptfile : rhdqzf
dataset    : None

In [6]:
# Set the required dataset on the `prefs` dictionary.
context.prefs["dataset"] = 62

In [7]:
# Scripts follow new form!!!

context.script[       "axfile"] = "mb50NCKFMASHTO"
context.script[         "with"] = phases_all  # replaces 'which'
context.script[     "inexcess"] = "mu q H2O"
context.script[       "dogmin"] = "1"                 # replaces 'dogmin yes 1'
context.script[       "maxvar"] = "6"                 # replaces 'setmaxvar'
context.script[    "diagramPT"] = "2 20 400 1100"     # new script
# context.script[        "calcP"] = "11"              # replaces 'setPwindow 11 11'
# context.script[        "calcT"] = "600"             # replaces 'setTwindow 600 600'
context.script["pseudosection"] = ""                  # replaces 'pseudosection yes'
context.script[   "samecoding"] = ["mu pa", "sp mt"]  # 

In [8]:
# Set up the chemical system:

# The oxides and their order
oxidestr =  "                              H2O        SiO2       Al2O3          CaO        MgO         FeO         K2O        Na2O        TiO2            O"     
oxlist = oxidestr.split()

# CHECK user oxide list against global THERMOCALC oxides/oxide order in datasets > oxide_data.txt???



# Construct initial bulk composition by creating the initial rbi object, 
# 1. set the oxide list:
rbi_sec = tc.rbi(oxides=oxidestr)
# 2. add the phases one by one (strings and floats can generally be used interchangeably):
#
#                (  "phase", "mode",  "  oxide1      oxide2         ...                                           " )
rbi_sec.add_phase(  "g",  "0.02"    , "       0    3.000000    0.984049    0.230147    0.395429    2.406326           0           0           0    0.015951")
rbi_sec.add_phase( "mu",  "0.159799", "1.000000    3.102219    1.393080    0.004483    0.058473    0.053149    0.386538    0.111220           0    0.002460")
rbi_sec.add_phase( "pa",  "0.006915", "       1    2.984010    1.505704    0.019628    0.001993    0.002589    0.033770    0.456416           0    0.000473")
rbi_sec.add_phase( "bi",  "0.090237", "0.932353    2.842048    0.636750           0    1.244448    1.572357    0.500000           0    0.067647    0.021202")
rbi_sec.add_phase("ilmm",  "0.004242", "      0           0           0           0    0.011126    1.032649           0           0    0.956225    0.043775")
rbi_sec.add_phase(  "q",  "0.240992", "       0           1           0           0           0           0           0           0           0           0")
rbi_sec.add_phase("H2O",  "0.458826", "       1           0           0           0           0           0           0           0           0           0")

# Alternatively, to specify the initial bulk composition as a single vector of 
# oxides rather than by phase, use simply:
# rbi_sec.add_phase(  "bulk", "1",  "  oxide1      oxide2         ...                                           " )  

# For help, run
# help(rbi_sec.add_phase)

# Add rbi object to scripts
context.script["rbi"] = rbi_sec

In [9]:
# Let's look at the initial chemistry of the system:
ox0 = system_components(oxlist, context.script["rbi"], phases="all")[1]
ox0_sys = [ 100*i/sum(ox0) for i in ox0 ]
atoms0_sys = atoms_from_ox(ox0_sys)
print("no. atoms in initial system: ", atoms0_sys)
print("oxide units (%) in initial system:\n", ox0_sys)

no. atoms in initial system:  303.9233143888544
oxide units (%) in initial system:
 [51.033958160128044, 37.06795159048513, 4.556744950605187, 0.08387182544455668, 1.8546201585534432, 3.152875948580158, 1.5539888315392187, 0.3087582008370118, 0.33782654393834355, 0.049403789888916846]


In [10]:
# Now create xyzguess section: Method 1: Manual xyzguess entry
# We use the `xyz` object this time. 
# It is really just a Python `OrderedDict`.
# First create a dictionary to record values
xyzguess = tc.xyz()
# Now add values
xyzguess[    "x(g)"] = "0.885508"
xyzguess[    "z(g)"] = "0.211255"
xyzguess[    "f(g)"] = "0.070810"
xyzguess[   "x(bi)"] = "0.579607"
xyzguess[   "y(bi)"] = "0.140313"
xyzguess[   "f(bi)"] = "0.126258"
xyzguess[   "t(bi)"] = "0.072042"
xyzguess[   "Q(bi)"] = "0.121282"
xyzguess[   "x(mu)"] = "0.461998"
xyzguess[   "y(mu)"] = "0.944788"
xyzguess[   "f(mu)"] = "0.009495"
xyzguess[   "n(mu)"] = "0.253509"
xyzguess[   "c(mu)"] = "0.016160"
xyzguess[   "x(pa)"] = "0.461998"
xyzguess[   "y(pa)"] = "0.996868"
xyzguess[   "f(pa)"] = "0.002376"
xyzguess[   "n(pa)"] = "0.955475"
xyzguess[   "c(pa)"] = "0.038175"
xyzguess[   "x(ma)"] = "0.461998"
xyzguess[   "y(ma)"] = "0.964236"
xyzguess[   "f(ma)"] = "0.009701"
xyzguess[   "n(ma)"] = "0.054672"
xyzguess[   "c(ma)"] = "0.944410"
xyzguess[  "x(chl)"] = "0.473331"
xyzguess[  "y(chl)"] = "0.520964"
xyzguess[  "f(chl)"] = "0.248348"
xyzguess["QAl(chl)"] = "0.230540  range -1.000 1.000"
xyzguess[ "Q1(chl)"] = "0.077845  range -1.000 1.000"
xyzguess[ "Q4(chl)"] = "0.092301  range -1.000 1.000"
xyzguess[   "f(ep)"] = "0.264635"
xyzguess[   "Q(ep)"] = "0.250072  range  0.000 0.500"
xyzguess[  "i(ilmm)"] = "0.829808"
xyzguess[  "g(ilmm)"] = "0.017481"
xyzguess[  "Q(ilmm)"] = "0.760053  range -0.990 0.990"
xyzguess[  "x(mt1)"] = "0.762081"
xyzguess[  "Q(mt1)"] = "0.808179"

context.script["xyzguess"] = xyzguess


# Method 2: Elsewhere, e.g. in ecrg-frac-xtn.ipynb, I've read these in from a text file

### Run

In [11]:
modes = []
atoms_sys_pre_solve = atoms0_sys
ox_sys = ox0_sys
atoms_lost = 0
ox_sys = [ 0 for i in range(len(oxlist)) ]

step = 0
for P,T in PT_steps:
    # set required windows
    context.script["calcP"] = P
    context.script["calcT"] = T
    
    # execute
    results = context.execute()
    modes.append((results.modes,step))
    
    # set resultant rbi back on script
    print("Executing for P,T = {},{}".format(P,T))
    context.script["rbi"] = results.rbi
    rbi_step = results.rbi
    
    # remove required phases; note cumulative changes to system chemistry    
    phases_this_step = list(context.script["rbi"].keys())
    phases_kept = [ i for i in phases_this_step if i not in list(removal_fraction.keys()) ]
    oxk0 = system_components(oxlist,rbi_step, phases_kept)[1]
    oxk = [ atoms_sys_pre_solve*i for i in oxk0 ]
    atoms_sys_post_solve = atoms_from_ox(oxk0)*atoms_sys_pre_solve
    
    for phase, removal_frac in removal_fraction.items():
        if phase in context.script["rbi"].keys():
            phase_post_solve = context.script["rbi"][phase]["mode"] 
            context.script["rbi"][phase]["mode"] *= (1.-removal_frac)
            print("Phase {} before = {} after = {}".format(phase, phase_post_solve, context.script["rbi"][phase]["mode"] ))            
            # get rid of it here too for viz purposes
                
    print("Atoms in system before = {} after = {}".format(atoms_sys_pre_solve, atoms_sys_post_solve ))
    oxd = { oxlist[i] : oxk[i] for i in range(len(oxlist)) }
    print("Oxides in system: sum: ", sum(oxk), "; values:")
    print(oxd)
    atoms_sys_pre_solve = atoms_sys_post_solve
    print("")
                
    # record new values to new dict for viz purposes
    final_modes = {}
    for key,val in context.script["rbi"].items():
        final_modes[key] = val["mode"]
    modes.append((final_modes,step))
    step += 1

Executing for P,T = 11.0,600
Phase g before = 0.02123 after = 0.0
Atoms in system before = 303.9233143888544 after = 297.471022424379
Oxides in system: sum:  97.73085682094641 ; values:
{'H2O': 51.033982703634855, 'SiO2': 36.100117504298325, 'Al2O3': 4.2395504249868745, 'CaO': 0.015328823904411818, 'MgO': 1.7281789842677726, 'FeO': 2.3691612717613, 'K2O': 1.5539886351852756, 'Na2O': 0.3087613489296371, 'TiO2': 0.33779721972770566, 'O': 0.043989904250260274}

Executing for P,T = 12.0,630
Phase g before = 0.002922 after = 0.0
Atoms in system before = 297.471022424379 after = 296.6018120968549
Oxides in system: sum:  97.42509468148587 ; values:
{'H2O': 51.033984933439584, 'SiO2': 35.969744203484225, 'Al2O3': 4.196857354498363, 'CaO': 0.009227460696107697, 'MgO': 1.7091309297089405, 'FeO': 2.2623866470581726, 'K2O': 1.5539828199266383, 'Na2O': 0.3087621275720646, 'TiO2': 0.33779739553371346, 'O': 0.043220809568051986}

Executing for P,T = 12.2,650
Phase g before = 0.000354 after = 0.0
Atom

In the above, I think THERMOCALC is blowing up because it's being fed a mode for garnet of 2.7800000000000022e-05. I think it's simply interpreting this as "too long" and bombing, rather than reading it as a hair-raising 2.780000 (this is probably the better outcome). 

- Keeping records step-by-step?
- Facilitating restart in event of slow THERMOCALC runs with crash mid-path?
- Check for missing input files at every step